In [283]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['figure.facecolor'] = 'white'

In [284]:
import os
import numpy as np
import pandas as pd

In [285]:
import requests
from requests.structures import CaseInsensitiveDict

In [ ]:
from dotenv import dotenv_values

config = dotenv_values(".env")  # config = {"USER": "foo", "EMAIL": "foo@example.org"}

In [286]:
BEARER_TOKEN = config.BEARER_TOKEN

In [287]:
url1 = "https://api.twitter.com/2/users/by/username/LulaOficial" 
nome_candidato = 'LulaOficial' # 2670726740

# url1 = "https://api.twitter.com/2/users/by/username/jairbolsonaro" # 128372940
# nome_candidato = 'jairbolsonaro'

headers = CaseInsensitiveDict()
headers["authorization"] = f"Bearer {BEARER_TOKEN}"

In [288]:
# Get userid based on username
response = requests.get(url1, headers=headers)
data = response.json()

In [289]:
user_id = data['data']['id']
user_id

'2670726740'

In [290]:
# Parameters
start_time = '2022-01-01T00:00:00.00-03:00'
end_time = '2022-06-30T00:00:00.00-03:00'

# start_time = '2022-01-01T00:00:00.00-03:00'
# end_time = '2022-01-02T00:00:00.00-03:00'

In [291]:
base_url = f"https://api.twitter.com/2/users/{user_id}/tweets?" \
    f"expansions=entities.mentions.username,author_id,referenced_tweets.id,attachments.media_keys&" \
    f"tweet.fields=created_at,author_id,lang,public_metrics,referenced_tweets,attachments&" \
    f"media.fields=preview_image_url,url,type&" \
    f"start_time={start_time}&end_time={end_time}&max_results=100"

In [292]:
def get_media_referenced(media_referenced_tweets_id):

    if media_referenced_tweets_id == "":
        return

    url_single = f"https://api.twitter.com/2/tweets/{media_referenced_tweets_id}?" \
        f"tweet.fields=lang&expansions=attachments.media_keys&media.fields=preview_image_url,url"
        
    response = requests.get(url_single, headers=headers)
    data = response.json()
    
    if 'includes' in data.keys():
        data = data['includes']
        if 'media' in data.keys():
            return data['media'][0]['type']

    return 


In [293]:
tweets_dic = {"id": [], "created_at": [], "text": [], 
"retweet_count": [], "reply_count": [], "like_count": [], "quote_count": [], 
"lang": [], "referenced_tweets_type": [], "referenced_tweets_id": [], 'referenced_tweets_qty': [], 'referenced_tweets_author': [],
'media_key': [], 'media_qty': [], 'media_type': [],
'mentions': [], 'qtd_mencoes': []}

next_token = ""

while True:

    url = f"{base_url}&pagination_token={next_token}" if next_token else base_url
    
    response = requests.get(url, headers=headers)
    data = response.json()

    tweets = data['data']

    for tweet in tweets:
        tweets_dic['id'].append(tweet['id'])
        tweets_dic['created_at'].append(tweet['created_at'])
        tweets_dic['text'].append(tweet['text'])
        tweets_dic['retweet_count'].append(tweet['public_metrics']['retweet_count'])
        tweets_dic['reply_count'].append(tweet['public_metrics']['reply_count'])
        tweets_dic['like_count'].append(tweet['public_metrics']['like_count'])
        tweets_dic['quote_count'].append(tweet['public_metrics']['quote_count'])

        tweets_dic['lang'].append(tweet.get('lang', ''))


        referenced_tweets_id = '' 
        author_referenced_tweet = ''
        if 'referenced_tweets' in tweet.keys():
            tweets_dic['referenced_tweets_type'].append(tweet['referenced_tweets'][0]['type'])
            referenced_tweets_id = tweet['referenced_tweets'][0]['id']
            tweets_dic['referenced_tweets_id'].append(referenced_tweets_id)
            tweets_dic['referenced_tweets_qty'].append(len(tweet['referenced_tweets']))

            if tweet['referenced_tweets'][0]['type'] == 'retweeted':
                author_referenced_tweet = tweet['text'][4:].split(':')[0]

            else:
                author_referenced_tweet = [ tweet_includes['author_id'] for tweet_includes in data['includes']['tweets'] if tweet_includes['id'] == tweet['referenced_tweets'][0]['id']]
                if not author_referenced_tweet:
                    author_referenced_tweet = 0
                else:
                    username =  [ user_includes['username'] for user_includes in data['includes']['users'] if user_includes['id'] == author_referenced_tweet[0] ]
                    if len(username):
                        author_referenced_tweet = username[0]
                    else:
                        author_referenced_tweet = author_referenced_tweet[0]

            tweets_dic['referenced_tweets_author'].append(author_referenced_tweet)

        else:
            tweets_dic['referenced_tweets_type'].append('')
            tweets_dic['referenced_tweets_id'].append('')
            tweets_dic['referenced_tweets_qty'].append(0)

            tweets_dic['referenced_tweets_author'].append('')


        if 'attachments' in tweet.keys():
            media_key = tweet['attachments']['media_keys'][0]
            tweets_dic['media_key'].append(media_key)
            tweets_dic['media_qty'].append(len(tweet['attachments']['media_keys']))

            media_type = [ media_includes['type'] for media_includes in data['includes']['media'] if media_includes['media_key'] == media_key ][0]
            tweets_dic['media_type'].append(media_type)
            
        else:

            media_type_referenced = ''
            if author_referenced_tweet != nome_candidato:
                media_type_referenced = get_media_referenced(referenced_tweets_id)

            if media_type_referenced:
                tweets_dic['media_key'].append('')
                tweets_dic['media_qty'].append(1)
                tweets_dic['media_type'].append(media_type_referenced)
            else:
                tweets_dic['media_key'].append('')
                tweets_dic['media_qty'].append(0)
                tweets_dic['media_type'].append('')

        if 'entities' in tweet.keys():
            mentions = [mention['username'] for mention in tweet['entities']['mentions']]
            tweets_dic['mentions'].append(", ".join(mentions))
            tweets_dic['qtd_mencoes'].append(  len(tweet['entities']['mentions'])  )
        else:
            tweets_dic['mentions'].append("")
            tweets_dic['qtd_mencoes'].append(0)


    next_token = data['meta'].get('next_token','')

    if next_token == "":
        break

In [295]:
df1 = pd.DataFrame(tweets_dic).set_index("id").sort_index()
# .sort_values(by='created_at')

In [296]:
df1.shape

(1460, 16)

In [297]:
dic_depth = {}

for index, row in df1.iterrows():
     this_depth = 0
     if row['referenced_tweets_id']:
          if row['referenced_tweets_id'] in dic_depth.keys():
               this_depth = dic_depth[row['referenced_tweets_id']] + 1

     dic_depth[index] = this_depth

df1['thread_count'] = dic_depth.values()

In [298]:
df1[df1['referenced_tweets_type']=='replied_to'][['created_at', 'text', 'like_count', 'referenced_tweets_id', 'referenced_tweets_type', 'referenced_tweets_author', 'thread_count', 'media_type', 'mentions']]

,created_at,text,like_count,referenced_tweets_id,referenced_tweets_type,referenced_tweets_author,thread_count,media_type,mentions
id,,,,,,,,,
1478117280827510795,2022-01-03T21:33:16.000Z,https://t.co/GosonhQxkN,4848,1478117272921194501,replied_to,LulaOficial,1,photo,
1478117285919440899,2022-01-03T21:33:17.000Z,https://t.co/GEJWIBW4oc,5540,1478117280827510795,replied_to,LulaOficial,2,photo,
1478117291464269826,2022-01-03T21:33:19.000Z,https://t.co/6eqarHYhaU,5105,1478117285919440899,replied_to,LulaOficial,3,photo,
1478117298263281667,2022-01-03T21:33:20.000Z,https://t.co/ZRybxe4I34,5463,1478117291464269826,replied_to,LulaOficial,4,photo,
1478117304697298948,2022-01-03T21:33:22.000Z,https://t.co/ILGrqYFkRk,4390,1478117298263281667,replied_to,LulaOficial,5,photo,
...,...,...,...,...,...,...,...,...,...
1539718067189694464,2022-06-22T21:12:48.000Z,"As 121 diretrizes, debatidas por representante...",1111,1539718065713254406,replied_to,LulaOficial,6,,
1541774112183435264,2022-06-28T13:22:47.000Z,@ricardostuckert Já acompanha o Lula no tiktok...,1678,1541739906657140738,replied_to,LulaOficial,1,,ricardostuckert
1542136363952361472,2022-06-29T13:22:15.000Z,@PauloVieiraReal A gente precisa conversar sob...,15545,1542117802257039364,replied_to,PauloVieiraReal,0,,PauloVieiraReal


In [299]:
dic_ext_links = {}

for index, row in df1.iterrows():
     text = row['text']
     # Misturas de quote com media type pode não pegar direito...
     if ("https://" in text or "https://" in  text) and (not text.startswith("http") ) and row['media_type'] == '' and row['referenced_tweets_type'] != 'quoted':
          dic_ext_links[index] = 1
     else:
          dic_ext_links[index] = 0

In [300]:
df1['ext_links'] = dic_ext_links.values()


In [301]:
dic_tipo_tweet = {}

for index, row in df1.iterrows():
     if row['referenced_tweets_type'] == "":
          dic_tipo_tweet[index] = 'root'
     elif row['referenced_tweets_type'] == "replied_to" and row['referenced_tweets_author'] != nome_candidato  :
          dic_tipo_tweet[index] = 'replied_to_other'
     elif row['referenced_tweets_type'] == "replied_to" and row['referenced_tweets_author'] == nome_candidato  :
          dic_tipo_tweet[index] = 'replied_to_self'
     else:
          dic_tipo_tweet[index] = row['referenced_tweets_type']

In [302]:
df1['tweet_type'] = dic_tipo_tweet.values()

In [303]:
df1['tweet_type'].value_counts()

root                1182
replied_to_self      153
retweeted             62
quoted                39
replied_to_other      24
Name: tweet_type, dtype: int64

In [304]:
dic_qtd_mencoes = {}

for index, row in df1.iterrows():
     if row['referenced_tweets_type'] == "":
          dic_qtd_mencoes[index] = 'root'
     elif row['referenced_tweets_type'] == "replied_to" and row['referenced_tweets_author'] != nome_candidato  :
          dic_qtd_mencoes[index] = 'replied_to_other'
     elif row['referenced_tweets_type'] == "replied_to" and row['referenced_tweets_author'] == nome_candidato  :
          dic_qtd_mencoes[index] = 'replied_to_self'
     else:
          dic_qtd_mencoes[index] = row['referenced_tweets_type']


In [305]:
# df1.to_pickle("./bolsonaro_2022_Q12.pkl")
df1.to_pickle("./lula_2022_Q12.pkl")